In [10]:
from main import get_miner
from log_parsing import from_id_to_template
import joblib
from data_integrate import *
from pattern_miner import *
import pattern_ranker as pr

In [2]:
log_template_miner = get_miner()

error ids: 

In [3]:
res_dict = joblib.load('output/fault_free_eventgraphs.joblib')

In [4]:
res_dict2 = joblib.load('output/fault_suffering_eventgraphs.joblib')

In [21]:


def extract_expected_events(results, fault_free_pattern):
    for r in results:
        key = r['events']
        for key_faultfree in fault_free_pattern.keys():
            if key.split('_')[0] == key_faultfree.split('_')[0]:
                if key.split('_')[1] != key_faultfree.split('_')[1]:
                    if not any([r_entry['events'] == key_faultfree for r_entry in results]):
                        r['events_expected'] = key_faultfree
                break
    return results

In [22]:
def abnormal_pattern_ranker_custom(normal_pattern_dict, abnormal_pattern_dict):
    score_dict = {}
    for key in abnormal_pattern_dict.keys():
        if key in normal_pattern_dict.keys():
            score = (2.0 * abnormal_pattern_dict[key] / (abnormal_pattern_dict[key] + normal_pattern_dict[key]) - 1)
            if score != 0:
                score_dict[key] = score
        else:
            score_dict[key] = 1.0

    return score_dict

global_abnormal_patterns = {}

def pattern_ranker_custom(fault_free_pattern, fault_suffering_event_graphs, fault_suffering_pattern, log_template_miner, topk=10, min_diff=1):
    actual_comparedto_faultfree = abnormal_pattern_ranker_custom(
        fault_free_pattern, fault_suffering_pattern)
    error_eventids = [185, 187, 188, 189, 201, 204, 205, 207, 208, 209]
    
    patterns_noerror =  {
        k: v for k, v in actual_comparedto_faultfree.items() 
        if not any(str(id_) in k.split('_') for id_ in error_eventids)
    }
    
    patterns_error =  {
        k: v for k, v in actual_comparedto_faultfree.items() 
        if any(str(id_) in k.split('_') for id_ in error_eventids)
    }

    reuslts_noerror = pattern_ranker_fromscore(patterns_noerror, fault_suffering_event_graphs, log_template_miner, topk)
    reuslts_error = pattern_ranker_fromscore(patterns_error, fault_suffering_event_graphs, log_template_miner, topk)
    reuslts_noerror = extract_expected_events(reuslts_noerror, fault_free_pattern)
    reuslts_error = extract_expected_events(reuslts_error, fault_free_pattern)
    
    return reuslts_noerror[0:topk], reuslts_error

def pattern_ranker_fromscore(abnormal_pattern_score, fault_suffering_event_graphs, log_template_miner, topk=10):
    move_list = set()
    for key in abnormal_pattern_score.keys():
        # only consider the root of child graph
        log_var = from_id_to_template(int(key.split("_")[1]),log_template_miner)
        if "Cpu" not in log_var and "Network" not in log_var and "Memory" not in log_var:
            for key1 in abnormal_pattern_score.keys():
                if int(key.split("_")[0]) == int(key1.split("_")[1]) and abs(abnormal_pattern_score[key]) <= abs(abnormal_pattern_score[key1]):
                    move_list.add(key)
    for item in move_list:
        abnormal_pattern_score.pop(item)

    result_list = []
    deepth_dict = {}
    for key, value in abnormal_pattern_score.items():
        deepth, pod = get_event_depth_pod(fault_suffering_event_graphs, key)
        
        if pod not in deepth_dict:
            deepth_dict[pod] = {}
        if key not in deepth_dict[pod]:
            deepth_dict[pod][key] = deepth
        elif deepth_dict[pod][key] < deepth:
            deepth_dict[pod][key] = deepth

        if pod == "":
            deepth = 1
            pod = "frontend-579b9bff58-t2dbm"

        result_list.append({"events": key, "score": value,
                                "deepth": deepth, "pod": pod})

    # if many alarm in one service instane, only persistent the deepest one
    move_list = set()
    alarm_eventids=['184', '92']
    for key_pod in deepth_dict.keys():
        pod_events = deepth_dict[key_pod]
            # Filter keys that contain any of the target ids
        matching_keys = {k: v for k, v in pod_events.items() 
                    if any(id in k for id in alarm_eventids)}
    
        # If no matches found, return None
        if not matching_keys:
            continue

        # Return the key with maximum depth value
        deepest_key = max(matching_keys.items(), key=lambda x: x[1])[0]
            # Create move list with all matching keys except the deepest
        for key in matching_keys.keys():
            if matching_keys[key] < matching_keys[deepest_key]:
                move_list.add(key)

    result_list = [d for d in result_list if d['events'] not in move_list]

    # if score is the same, deeper is prefer
    result_list = sorted(result_list, key=lambda i: (
        abs(i['score']), i['deepth']), reverse=True)
    
    return result_list[0:topk]

In [5]:
fault_free_pattern = get_pattern_support(res_dict['2022-08-22 03:51'])

In [6]:
res_dict2['2022-08-22 05:02']['inject_pod']

'checkoutservice-578fcf4766-9csqn'

In [7]:
fault_suffering_event_graphs = res_dict2['2022-08-22 05:02']['event_graphs']
fault_suffering_pattern = get_pattern_support(fault_suffering_event_graphs)

In [23]:
r = pattern_ranker_custom(fault_free_pattern, fault_suffering_event_graphs, fault_suffering_pattern, log_template_miner)

In [25]:
r[1]

[{'events': '2_185',
  'score': 1.0,
  'deepth': 1,
  'pod': 'frontend-579b9bff58-t2dbm',
  'events_expected': '2_82'},
 {'events': '153_188',
  'score': 1.0,
  'deepth': 1,
  'pod': 'frontend-579b9bff58-t2dbm',
  'events_expected': '153_154'}]